In [2]:
import numpy as np

In [31]:
test=np.random.random((1001,100,3))
test2=np.random.random((1001,3))
test3=np.stack([test2 for i in range(100)],axis=1)
print(test3.shape)
rest=np.transpose((np.transpose(test,axes=[1,0,2])-test2),axes=[1,0,2])
rest=test-test3

(1001, 100, 3)


In [35]:
test3.shape

(1001, 100, 3)

In [15]:
(np.transpose(test,axes=[1,0,2])-test2).shape

(100, 1001, 3)

In [18]:
test[:,0,:]

array([[0.46071067, 0.49387381, 0.65788116],
       [0.85211678, 0.6030276 , 0.49031854],
       [0.69293535, 0.62295761, 0.50646184],
       ...,
       [0.9570016 , 0.51278733, 0.73022375],
       [0.70286892, 0.83668662, 0.60222445],
       [0.6245749 , 0.65061398, 0.79533664]])

In [37]:
test[:,0,:]-test2[:,:]-rest[:,0,:]

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       ...,
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [19]:
rest[:,0,:]

array([[-0.53695928, -0.16410298, -0.01520142],
       [ 0.54613555,  0.4736633 ,  0.21400749],
       [ 0.38279329,  0.10880986,  0.37159959],
       ...,
       [ 0.22950276, -0.23917028,  0.50917345],
       [-0.10398839,  0.08555296,  0.03512456],
       [-0.21092282, -0.00552054, -0.04465297]])

In [12]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from scipy.stats import gaussian_kde
import numpy as np

%matplotlib qt

class CelestialBody:
    """
    Represents a celestial body (e.g., moon or test particle) with attributes such as name, color, position, and trail options.

    Attributes:
        name (str): Name of the celestial body.
        color (str): Color used for plotting the body.
        pos (ndarray): 3D positional data (timesteps x 3).
        trail (bool): Whether the body should have a trailing line when animated.
    """
    def __init__(self, name, color, pos, trail=False):
        self.name = name
        self.color = color
        self.pos = pos  # Positional data as a 3D NumPy array (timesteps x 3)
        self.trail = trail


class Dataset:
    """
    Handles the simulation dataset, including reading the header, extracting positional data,
    and initializing celestial bodies for visualization.

    Attributes:
        filepath (str): Path to the binary simulation data file.
        header (dict): Header information extracted from the file.
        num_moons (int): Number of moons in the dataset.
        num_test_particles (int): Number of test particles in the dataset.
        positions (ndarray): 3D array of positions and velocities for all objects (timesteps x objects x 6).
        moons (list): List of CelestialBody objects representing moons.
        test_particles (list): List of CelestialBody objects representing test particles.
    """
    def __init__(self, filepath):
        self.filepath = filepath
        self.header = self._read_header()
        self.num_moons = int(self.header['Moon Count'])
        self.num_test_particles = int(self.header['Number of Test Particles'])
        self.positions = self._read_binary_file()
        self.moons, self.test_particles = self._initialize_bodies()

    def _read_header(self):
        """
        Reads and parses the header section of the binary file.

        Returns:
            dict: Parsed header as a dictionary.
        """
        header = {}
        with open(self.filepath, 'rb') as file:
            while True:
                line = file.readline()
                if b"End of Header" in line:
                    break
                key, value = line.decode('utf-8').strip().split(':', 1)
                header[key.strip()] = self._convert_value(value.strip())
        return header

    def _convert_value(self, value):
        """
        Converts header values to their appropriate data types.

        Args:
            value (str): Header value to be converted.

        Returns:
            int, float, or str: Converted value.
        """
        try:
            if '.' in value:
                return float(value)
            return int(value)
        except ValueError:
            return value

    def _read_binary_file(self):
        """
        Reads the binary data section of the file, skipping the header.

        Returns:
            ndarray: Reshaped positional and velocity data (timesteps x objects x 6).
        """
        length_header = len(self._read_header_raw()) + len("End of Header\n")
        data = np.fromfile(self.filepath, dtype=np.float64, offset=length_header)
        return data.reshape(-1, self.num_moons + self.num_test_particles, 6)

    def _read_header_raw(self):
        """
        Reads the raw header for calculating its length.

        Returns:
            bytes: Raw header data.
        """
        header = []
        with open(self.filepath, 'rb') as file:
            while True:
                line = file.readline()
                if b"End of Header" in line:
                    break
                header.append(line)
        return b"".join(header)

    def _initialize_bodies(self):
        """
        Initializes celestial bodies (moons and test particles) with positional data.

        Returns:
            tuple: A list of moon CelestialBody objects and test particle CelestialBody objects.
        """
        moon_names = self.header['Moon Names'].split(', ')
        body_colors = ['yellow', 'red', 'chartreuse', 'lightblue', 'orange', 'brown', 'blue', 'pink', 'red', 'black',
                       'green', 'purple', 'cyan', 'magenta', 'gold', 'silver', 'lime', 'navy', 'maroon', 'crimson']
        moon_colors = body_colors[:self.num_moons]

        moons = [
            CelestialBody(moon_names[i], moon_colors[i], self.positions[:, i, :3]-self.positions[:,0,:3], trail=True)
            for i in range(self.num_moons)
        ]# 0 should be saturn, so position relative to saturn
        test_particle_positions = np.ones((self.positions.shape[0],self.positions.shape[1]-self.num_moons,3))
        for i in range(test_particle_positions.shape[1]):
            test_particle_positions[:,i,:] = self.positions[:, self.num_moons+i, :3]#-self.positions[:,0,:3]
        # test_particle_positions = self.positions[:, self.num_moons:, :3]-np.stack([self.positions[:,0,:3] for i in range(self.positions.shape[1]-self.num_moons)],axis=1) # 0 should be saturn, so position relative to saturn
        test_particles = [
            CelestialBody(str(i), "navy", test_particle_positions[:, i, :], trail=False)
            for i in range(test_particle_positions.shape[1])
        ]

        return moons, test_particles


class AnimationManager:
    def __init__(self, dataset):
        self.dataset = dataset

    def plot_2d(self, coords=[0, 1], n_farthest_filter=10, big_traillength=100, small_traillength=3, frame_time=10,interval=1):
        """
        Animates the simulation, displaying the positions and trails of celestial bodies.

        Args:
            coords (list): Coordinate indices to plot (e.g., [0, 1] for X-Y plane).
            n_farthest_filter (int): Number of moons to exclude from zoomed view based on distance.
            big_traillength (int): Length of the main trail for moons.
            small_traillength (int): Length of the zoomed trail for moons.
            interval (int): Time interval between animation frames.
        """
        # Create a figure with two subplots
        figure, (ax, ax2) = plt.subplots(1, 2, figsize=(12, 6))

        # Set axis limits for the full view
        xmax, xmin = np.max([np.max(body.pos[:, coords[0]]) for body in self.dataset.moons]), np.min([np.min(body.pos[:, coords[0]]) for body in self.dataset.moons])
        ymax, ymin = np.max([np.max(body.pos[:, coords[1]]) for body in self.dataset.moons]), np.min([np.min(body.pos[:, coords[1]]) for body in self.dataset.moons])
        ax.set_xlim(xmin - 0.1 * abs(xmin), xmax + 0.1 * abs(xmax))
        ax.set_ylim(ymin - 0.1 * abs(ymin), ymax + 0.1 * abs(ymax))
        ax.set_aspect('equal', adjustable='box')

        # Filter moons for the zoomed-in view
        nth_largest_indices = np.argsort([np.min(body.pos[:, 0]**2 + body.pos[:, 1]**2) for body in self.dataset.moons])[:-n_farthest_filter]
        filtered_moons = [self.dataset.moons[i] for i in nth_largest_indices]
        ax2.set_xlim(-3e8, 3e8)
        ax2.set_ylim(-3e8, 3e8)
        ax2.set_aspect('equal', adjustable='box')

        # Initialize plot objects for moons and test particles
        moon_lines = {body: ax.plot([], [], label=f"{body.name}", color=body.color, linestyle='-')[0] for body in self.dataset.moons}
        moon_markers = {}
        zoom_lines = {}
        zoom_markers = {}

        # Special marker size for Saturn
        large_marker_size = 15
        default_marker_size = 6

        for body in self.dataset.moons:
            marker_size = large_marker_size if body.name.lower() == "saturn" else default_marker_size
            moon_markers[body] = ax.plot([], [], label=f"{body.name}", color=body.color, marker='o', linestyle='', markersize=marker_size)[0]
            if body in filtered_moons:
                zoom_lines[body] = ax2.plot([], [], label=f"{body.name}", color=body.color, linestyle='-')[0]
                zoom_markers[body] = ax2.plot([], [], label=f"{body.name}", color=body.color, marker='o', linestyle='', markersize=marker_size)[0]

        test_particle_line = ax2.plot([], [], ".", label="Test Particles", color="navy", markersize=1)[0]

        # Initialization function for the animation
        def init():
            for line in moon_lines.values():
                line.set_data([], [])
            for marker in moon_markers.values():
                marker.set_data([], [])
            for zoom_line in zoom_lines.values():
                zoom_line.set_data([], [])
            for zoom_marker in zoom_markers.values():
                zoom_marker.set_data([], [])
            test_particle_line.set_data([], [])
            return list(moon_lines.values()) + list(moon_markers.values()) + list(zoom_lines.values()) + list(zoom_markers.values()) + [test_particle_line]

        # Update function for each frame
        def update(i):
            for body in self.dataset.moons:
                moon_lines[body].set_data(
                    body.pos[max(i - big_traillength, 0):i + 1, coords[0]],  # Include current position
                    body.pos[max(i - big_traillength, 0):i + 1, coords[1]]
                )
                moon_markers[body].set_data(
                    body.pos[i, coords[0]],
                    body.pos[i, coords[1]]
                )
            for body in filtered_moons:
                zoom_lines[body].set_data(
                    body.pos[max(i - small_traillength, 0):i + 1, coords[0]],  # Include current position
                    body.pos[max(i - small_traillength, 0):i + 1, coords[1]]
                )
                zoom_markers[body].set_data(
                    body.pos[i, coords[0]],
                    body.pos[i, coords[1]]
                )
            test_particle_line.set_data(
                self.dataset.positions[i, self.dataset.num_moons:, coords[0]],
                self.dataset.positions[i, self.dataset.num_moons:, coords[1]]
            )
            return list(moon_lines.values()) + list(moon_markers.values()) + \
                list(zoom_lines.values()) + list(zoom_markers.values()) + [test_particle_line]


        # Create the animation and assign it to an attribute to prevent garbage collection
        self.animation = anim.FuncAnimation(
            figure,
            update,
            init_func=init,
            frames=np.arange(0, self.dataset.positions.shape[0], interval),
            interval=frame_time,
            blit=False
        )

        # Set axis labels
        labels = {0: 'X (m)', 1: 'Y (m)', 2: 'Z (m)'}
        ax.set_xlabel(labels[coords[0]])
        ax.set_ylabel(labels[coords[1]])
        ax2.set_xlabel(labels[coords[0]])
        ax2.set_ylabel(labels[coords[1]])

        # Add a legend
        marker_handles = [moon_markers[body] for body in self.dataset.moons]
        figure.legend(handles=marker_handles, loc='center right', bbox_to_anchor=(0.98, 0.5), title="Moons")

        # Adjust layout for better readability
        plt.subplots_adjust(left=0.05, right=0.85, top=0.95, bottom=0.05, wspace=0.15)

        plt.show()


    def plot_3d(self, n_farthest_filter=10, big_traillength=100, small_traillength=3, frame_time=100, interval=1):
        figure = plt.figure(figsize=(14, 7))

        # Create two 3D subplots
        ax = figure.add_subplot(121, projection='3d')
        ax2 = figure.add_subplot(122, projection='3d')

        # Set wide view axis limits
        xmax, xmin = np.max([np.max(body.pos[:, 0]) for body in self.dataset.moons]), np.min([np.min(body.pos[:, 0]) for body in self.dataset.moons])
        ymax, ymin = np.max([np.max(body.pos[:, 1]) for body in self.dataset.moons]), np.min([np.min(body.pos[:, 1]) for body in self.dataset.moons])
        zmax, zmin = np.max([np.max(body.pos[:, 2]) for body in self.dataset.moons]), np.min([np.min(body.pos[:, 2]) for body in self.dataset.moons])

        # Add debugging prints for axis limits
        # print(f"X Range: {xmin} to {xmax}")
        # print(f"Y Range: {ymin} to {ymax}")
        # print(f"Z Range: {zmin} to {zmax}")

        ax.set_xlim(xmin, xmax)
        ax.set_ylim(ymin, ymax)
        ax.set_zlim(zmin, zmax)

        # Zoomed-in view filter based on distance
        nth_largest_indices = np.argsort([np.min(body.pos[:, 0]**2 + body.pos[:, 1]**2 + body.pos[:, 2]**2) for body in self.dataset.moons])[:-n_farthest_filter]
        filtered_moons = [self.dataset.moons[i] for i in nth_largest_indices]

        # Zoomed-in axis limits
        zoom_range = 3e8
        ax2.set_xlim(-zoom_range, zoom_range)
        ax2.set_ylim(-zoom_range, zoom_range)
        ax2.set_zlim(-zoom_range, zoom_range)

        # Initialize moon plot objects
        moon_lines = {body: ax.plot([], [], [], label=f"{body.name}", color=body.color, linestyle='-')[0] for body in self.dataset.moons}
        moon_markers = {body: ax.plot([], [], [], label=f"{body.name}", color=body.color, marker='o', linestyle='')[0] for body in self.dataset.moons}

        zoom_lines = {body: ax2.plot([], [], [], label=f"{body.name}", color=body.color, linestyle='-')[0] for body in filtered_moons}
        zoom_markers = {body: ax2.plot([], [], [], label=f"{body.name}", color=body.color, marker='o', linestyle='')[0] for body in filtered_moons}

        # Initialize test particle plot objects
        test_particle_line = ax2.plot([], [], [], ".", label="Test Particles", color="navy", markersize=1)[0]

        def init():
            # Initialize moons
            for line in moon_lines.values():
                line.set_data_3d([], [], [])
            for marker in moon_markers.values():
                marker.set_data_3d([], [], [])
            for line in zoom_lines.values():
                line.set_data_3d([], [], [])
            for marker in zoom_markers.values():
                marker.set_data_3d([], [], [])

            test_particle_line.set_data_3d([], [], [])

            # Add debugging prints for init function
            #print("Init function called.")

            return list(moon_lines.values()) + list(moon_markers.values()) + \
                list(zoom_lines.values()) + list(zoom_markers.values()) + \
                [test_particle_line]

        def update(i):
            # Update moons in wide view
            for body in self.dataset.moons:
                moon_lines[body].set_data_3d(
                    body.pos[max(i - big_traillength, 0):i + 1, 0],  # Include current position
                    body.pos[max(i - big_traillength, 0):i + 1, 1],
                    body.pos[max(i - big_traillength, 0):i + 1, 2]
                )
                moon_markers[body].set_data_3d(
                    body.pos[i, 0],
                    body.pos[i, 1],
                    body.pos[i, 2]
                )

            # Update moons in zoomed-in view
            for body in filtered_moons:
                zoom_lines[body].set_data_3d(
                    body.pos[max(i - small_traillength, 0):i + 1, 0],  # Include current position
                    body.pos[max(i - small_traillength, 0):i + 1, 1],
                    body.pos[max(i - small_traillength, 0):i + 1, 2]
                )
                zoom_markers[body].set_data_3d(
                    body.pos[i, 0],
                    body.pos[i, 1],
                    body.pos[i, 2]
                )

            # Update test particles
            test_particle_line.set_data_3d(
                self.dataset.positions[i, self.dataset.num_moons:, 0],
                self.dataset.positions[i, self.dataset.num_moons:, 1],
                self.dataset.positions[i, self.dataset.num_moons:, 2]
            )

            return list(moon_lines.values()) + list(moon_markers.values()) + \
                list(zoom_lines.values()) + list(zoom_markers.values()) + \
                [test_particle_line]


        # Create animation
        self.animation = anim.FuncAnimation(
            figure,
            update,
            init_func=init,
            frames=np.arange(0, self.dataset.moons[0].pos.shape[0], interval),
            interval=frame_time,
            blit=False
        )

        # Set labels and legend
        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_zlabel('Z (m)')
        ax2.set_xlabel('X (m)')
        ax2.set_ylabel('Y (m)')
        ax2.set_zlabel('Z (m)')

        handles = [moon_markers[body] for body in self.dataset.moons]
        figure.legend(handles=handles, loc='center right', bbox_to_anchor=(0.98, 0.5), title="Moons")
        plt.subplots_adjust(left=0.05, right=0.85, top=0.95, bottom=0.05, wspace=0.2)

        plt.show()


    def plot_centered(self, moon_name, width, frame_time=10, trail_length=100, target_trail_length=150, interval=1):
        """Single-axis plot centered on a specific moon with no trails for test particles"""
        figure, ax = plt.subplots(figsize=(14, 7))
        target_moon = next((body for body in self.dataset.moons if body.name.lower() == moon_name.lower()), None)
        if not target_moon:
            raise ValueError(f"Moon '{moon_name}' not found!")

        ax.set_xlim(-width / 2, width / 2)
        ax.set_ylim(-width / 2, width / 2)
        ax.set_aspect('equal', adjustable='box')

        # Create a larger marker for the target moon (to make it more visible)
        moon_marker = ax.plot([], [], label=f"{target_moon.name}", color=target_moon.color, marker='o', markersize=8, linestyle='', zorder=5)[0]
        
        # Initialize trail for target moon
        target_trail, = ax.plot([], [], color=target_moon.color, linestyle='-', alpha=0.5, zorder=2)

        # Create markers and trails for other moons
        other_markers = {
            body: ax.plot([], [], label=f"{body.name}", color=body.color, marker='o', markersize=6, linestyle='', zorder=3)[0]
            for body in self.dataset.moons if body != target_moon
        }
        
        other_trails = {
            body: ax.plot([], [], color=body.color, linestyle='-', alpha=0.5, zorder=1)[0]
            for body in self.dataset.moons if body != target_moon
        }

        # Test particle marker without trails
        test_particle_marker = ax.plot([], [], ".", label="Test Particles", color="navy", markersize=1, zorder=0)[0]

        # Initialize the trail for the centered moon (target_moon)
        target_moon_trail = ax.plot([], [], color='red', linestyle='--', alpha=0.5, zorder=2)[0]

        def init():
            moon_marker.set_data([], [])
            target_trail.set_data([], [])
            target_moon_trail.set_data([], [])
            for marker in other_markers.values():
                marker.set_data([], [])
            for trail in other_trails.values():
                trail.set_data([], [])
            test_particle_marker.set_data([], [])
            return [moon_marker] + list(other_markers.values()) + [test_particle_marker] + list(other_trails.values()) + [target_trail, target_moon_trail]

        def update(i):
            # Update position for target moon marker
            center_x, center_y = target_moon.pos[i, 0], target_moon.pos[i, 1]
            ax.set_xlim(center_x - width / 2, center_x + width / 2)
            ax.set_ylim(center_y - width / 2, center_y + width / 2)
            moon_marker.set_data(center_x, center_y)

            # Update trail for the target moon
            trail_x = target_moon.pos[max(0, i - trail_length):i + 1, 0]
            trail_y = target_moon.pos[max(0, i - trail_length):i + 1, 1]
            target_trail.set_data(trail_x, trail_y)

            # Update trail for the centered target moon
            target_moon_trail_x = target_moon.pos[max(0, i - target_trail_length):i + 1, 0]
            target_moon_trail_y = target_moon.pos[max(0, i - target_trail_length):i + 1, 1]
            target_moon_trail.set_data(target_moon_trail_x, target_moon_trail_y)

            # Update positions and trails for the other moons
            for body, marker in other_markers.items():
                marker.set_data(body.pos[i, 0], body.pos[i, 1])
                trail_x = body.pos[max(0, i - trail_length):i + 1, 0]
                trail_y = body.pos[max(0, i - trail_length):i + 1, 1]
                other_trails[body].set_data(trail_x, trail_y)

            # Update test particles (positions only, no trails)
            test_particle_marker.set_data(
                self.dataset.positions[i, self.dataset.num_moons:, 0],
                self.dataset.positions[i, self.dataset.num_moons:, 1]
            )

            return [moon_marker] + list(other_markers.values()) + [test_particle_marker] + list(other_trails.values()) + [target_trail, target_moon_trail]

        self.animation = anim.FuncAnimation(
            figure,
            update,
            init_func=init,
            frames=np.arange(0, self.dataset.positions.shape[0], interval),
            interval=frame_time,
            blit=False
        )

        # Set labels and legend
        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')

        # Add a customized legend with the markers outside the plot
        marker_handles = [moon_marker] + list(other_markers.values()) + [test_particle_marker]
        figure.legend(handles=marker_handles, loc='center left', bbox_to_anchor=(0.7, 0.5), title="Bodies")

        plt.subplots_adjust(right=0.85)  # Adjust the plot area to make space for the legend

        plt.show()

    def plot_centered_3d(self, moon_name, width, frame_time=10, trail_length=100, target_trail_length=150, interval=1):
        """3D plot centered on a specific moon with no trails for test particles"""
        figure = plt.figure(figsize=(14, 7))
        ax = figure.add_subplot(111, projection='3d')

        # Find the target moon
        target_moon = next((body for body in self.dataset.moons if body.name.lower() == moon_name.lower()), None)
        if not target_moon:
            raise ValueError(f"Moon '{moon_name}' not found!")

        # Set initial limits for the 3D view
        ax.set_xlim(-width / 2, width / 2)
        ax.set_ylim(-width / 2, width / 2)
        ax.set_zlim(-width / 2, width / 2)

        # Create a larger marker for the target moon
        moon_marker = ax.plot([], [], [], label=f"{target_moon.name}", color=target_moon.color, marker='o', markersize=8, linestyle='', zorder=5)[0]

        # Initialize trail for the target moon
        target_trail, = ax.plot([], [], [], color=target_moon.color, linestyle='-', alpha=0.5, zorder=2)

        # Create markers and trails for other moons
        other_markers = {
            body: ax.plot([], [], [], label=f"{body.name}", color=body.color, marker='o', markersize=6, linestyle='', zorder=3)[0]
            for body in self.dataset.moons if body != target_moon
        }

        other_trails = {
            body: ax.plot([], [], [], color=body.color, linestyle='-', alpha=0.5, zorder=1)[0]
            for body in self.dataset.moons if body != target_moon
        }

        # Test particle marker without trails
        test_particle_marker = ax.plot([], [], [], ".", label="Test Particles", color="navy", markersize=1, zorder=0)[0]

        def init():
            moon_marker.set_data_3d([], [], [])
            target_trail.set_data_3d([], [], [])
            for marker in other_markers.values():
                marker.set_data_3d([], [], [])
            for trail in other_trails.values():
                trail.set_data_3d([], [], [])
            test_particle_marker.set_data_3d([], [], [])
            return [moon_marker] + list(other_markers.values()) + [test_particle_marker] + list(other_trails.values()) + [target_trail]

        def update(i):
            # Center the view on the target moon
            center_x, center_y, center_z = target_moon.pos[i, 0], target_moon.pos[i, 1], target_moon.pos[i, 2]
            ax.set_xlim(center_x - width / 2, center_x + width / 2)
            ax.set_ylim(center_y - width / 2, center_y + width / 2)
            ax.set_zlim(center_z - width / 2, center_z + width / 2)

            # Update position for target moon marker
            moon_marker.set_data_3d(center_x, center_y, center_z)

            # Update trail for the target moon
            trail_x = target_moon.pos[max(0, i - trail_length):i + 1, 0]
            trail_y = target_moon.pos[max(0, i - trail_length):i + 1, 1]
            trail_z = target_moon.pos[max(0, i - trail_length):i + 1, 2]
            target_trail.set_data_3d(trail_x, trail_y, trail_z)

            # Update positions and trails for the other moons
            for body, marker in other_markers.items():
                marker.set_data_3d(body.pos[i, 0], body.pos[i, 1], body.pos[i, 2])
                trail_x = body.pos[max(0, i - trail_length):i + 1, 0]
                trail_y = body.pos[max(0, i - trail_length):i + 1, 1]
                trail_z = body.pos[max(0, i - trail_length):i + 1, 2]
                other_trails[body].set_data_3d(trail_x, trail_y, trail_z)

            # Update test particles (positions only, no trails)
            test_particle_marker.set_data_3d(
                self.dataset.positions[i, self.dataset.num_moons:, 0],
                self.dataset.positions[i, self.dataset.num_moons:, 1],
                self.dataset.positions[i, self.dataset.num_moons:, 2]
            )

            return [moon_marker] + list(other_markers.values()) + [test_particle_marker] + list(other_trails.values()) + [target_trail]

        self.animation = anim.FuncAnimation(
            figure,
            update,
            init_func=init,
            frames=np.arange(0, self.dataset.positions.shape[0], interval),
            interval=frame_time,
            blit=False
        )

        # Set labels and legend
        ax.set_xlabel('X (m)')
        ax.set_ylabel('Y (m)')
        ax.set_zlabel('Z (m)')

        # Add a customized legend with the markers outside the plot
        marker_handles = [moon_marker] + list(other_markers.values()) + [test_particle_marker]
        figure.legend(handles=marker_handles, loc='center left', bbox_to_anchor=(0.75, 0.5), title="Bodies")

        plt.subplots_adjust(right=0.85)  # Adjust the plot area to make space for the legend

        plt.show()



    def plot_polar_cartesian(self, r_max, r_min=0,theta_max = 2*np.pi ,frame_time=10, trail_length=100, interval=1, heatmap=False):
        """Polar to Cartesian plot with (r, theta) coordinates for all moons, Saturn, and test particles, with an optional heatmap for test particles."""
        figure, ax = plt.subplots(figsize=(14, 7))
        
        # Set up the axis: r is on the x-axis, theta (0 to 2pi) is on the y-axis
        ax.set_xlim(r_min, r_max)
        ax.set_ylim(0, theta_max)
        ax.set_xlabel('Radial Distance (r) [m]')
        ax.set_ylabel('Angle (theta) [radians]')
        
        # Helper function to convert Cartesian (x, y) to polar (r, theta)
        def cartesian_to_polar(x, y):
            r = np.sqrt(x ** 2 + y ** 2)
            theta = np.arctan2(y, x)
            return r, np.mod(theta, 2 * np.pi)  # Ensure theta is in the range [0, 2pi]

        # Create markers and trails for all moons, including Saturn at the origin
        moon_markers = {
            body: ax.plot([], [], label=f"{body.name}", color=body.color, marker='o', markersize=6, linestyle='')[0]
            for body in self.dataset.moons
        }
        
        moon_trails = {
            body: ax.plot([], [], color=body.color, linestyle='-', alpha=0.5)[0]
            for body in self.dataset.moons
        }

        # Test particles (either as individual markers or as a heatmap)
        if not heatmap:
            test_particle_marker = ax.plot([], [], ".", label="Test Particles", color="navy", markersize=1)[0]
        else:
            # Create an empty 2D histogram for the heatmap (r and theta grid)
            heatmap_data, xedges, yedges = np.histogram2d([], [], bins=(100, 100), range=[[r_min, r_max], [0, 2 * np.pi]])
            heatmap_img = ax.imshow(heatmap_data.T, extent=[r_min, r_max, 0, 2 * np.pi], origin='lower', aspect='auto', cmap='viridis', alpha=0.8)

        # Saturn at origin (barycenter) with an "X" marker
        saturn_marker = ax.plot([0], [0], "x", label="Saturn (Barycenter)", color="yellow", markersize=10)[0]

        def init():
            # Initialize markers and trails to empty
            for marker in moon_markers.values():
                marker.set_data([], [])
            for trail in moon_trails.values():
                trail.set_data([], [])
            if not heatmap:
                test_particle_marker.set_data([], [])
                return list(moon_markers.values()) + [test_particle_marker] + list(moon_trails.values()) + [saturn_marker]
            else:
                heatmap_img.set_data(np.zeros_like(heatmap_img.get_array()))
                return list(moon_markers.values()) + list(moon_trails.values()) + [heatmap_img, saturn_marker]

        def update(i):
            # Update the positions and trails for all moons in polar coordinates
            for body, marker in moon_markers.items():
                r, theta = cartesian_to_polar(body.pos[i, 0], body.pos[i, 1])
                marker.set_data(r, theta)
                
                # Update the trail of the moons in polar coordinates
                trail_r, trail_theta = cartesian_to_polar(
                    body.pos[max(0, i - trail_length):i + 1, 0],
                    body.pos[max(0, i - trail_length):i + 1, 1]
                )
                moon_trails[body].set_data(trail_r, trail_theta)

            # Plot title:
            plt.title(f"frame {i}")
            
            if not heatmap:
                # Update the test particles in polar coordinates
                r_test, theta_test = cartesian_to_polar(
                    self.dataset.positions[i, self.dataset.num_moons:, 0],
                    self.dataset.positions[i, self.dataset.num_moons:, 1]
                )
                test_particle_marker.set_data(r_test, theta_test)

                return list(moon_markers.values()) + [test_particle_marker] + list(moon_trails.values()) + [saturn_marker]
            
            else:
                # Update the heatmap for test particles
                r_test, theta_test = cartesian_to_polar(
                    self.dataset.positions[i, self.dataset.num_moons:, 0],
                    self.dataset.positions[i, self.dataset.num_moons:, 1]
                )
                
                # Recreate the 2D histogram with current test particle positions
                heatmap_data, xedges, yedges = np.histogram2d(r_test, theta_test, bins=(100, 100), range=[[r_min, r_max], [0, 2 * np.pi]])
                
                # Update the heatmap image data
                heatmap_img.set_data(heatmap_data.T)
                heatmap_img.set_clim(vmin=0, vmax=np.max(heatmap_data))  # Adjust color limits to avoid full saturation

                return list(moon_markers.values()) + list(moon_trails.values()) + [heatmap_img, saturn_marker]

        # Create the animation
        self.animation = anim.FuncAnimation(
            figure,
            update,
            init_func=init,
            frames=np.arange(0, self.dataset.positions.shape[0], interval),
            interval=frame_time,
            blit=False
        )

        # Add a customized legend with the markers outside the plot
        if not heatmap:
            marker_handles = list(moon_markers.values()) + [test_particle_marker, saturn_marker]
        else:
            marker_handles = list(moon_markers.values()) + [saturn_marker]

        figure.legend(handles=marker_handles, loc='center left', bbox_to_anchor=(0.85, 0.5), title="Bodies")

        plt.subplots_adjust(right=0.85)  # Adjust the plot area to make space for the legend

        plt.show()
    
    def plot_polar_cartesian_with_z(self, r_max, r_min=0, z_min=None, z_max=None,elevation = 5 , azimuth = 90 ,frame_time=10, trail_length=100, interval=1, heatmap=False):
        """3D Polar to Cartesian plot with (r, theta, z) coordinates for all moons, Saturn, and test particles."""
        figure = plt.figure(figsize=(14, 7))
        ax = figure.add_subplot(111, projection='3d')  # Create a 3D subplot
        
        # Set the viewing angle (elevation and azimuth)
        ax.view_init(elev=elevation, azim=azimuth) 
    
        # Set up the axis: r is on the x-axis, theta (0 to 3pi) is on the y-axis, and z is on the z-axis
        ax.set_xlim(r_min, r_max)
        ax.set_ylim(-2*np.pi, 2 * np.pi)  # Extend theta axis from -2pi to 2pi
        if z_min is not None and z_max is not None:
            ax.set_zlim(z_min, z_max)  # Set z limits based on user input
        else:
            ax.set_zlim(-r_max, r_max)  # Default z limits based on r_max
        ax.set_xlabel('Radial Distance (r) [m]')
        ax.set_ylabel('Angle (theta) [radians]')
        ax.set_zlabel('Height (z) [m]')
        
        # Helper function to convert Cartesian (x, y) to polar (r, theta)
        def cartesian_to_polar(x, y):
            r = np.sqrt(x ** 2 + y ** 2)
            theta = np.arctan2(y, x)
            return r, np.mod(theta, 2 * np.pi)  # Ensure theta is in the range [0, 2pi]

        # Create markers and trails for all moons, including Saturn at the origin
        moon_markers = {
            body: ax.plot([], [], [], label=f"{body.name}", color=body.color, marker='o', markersize=6, linestyle='')[0]
            for body in self.dataset.moons
        }
        
        moon_trails = {
            body: ax.plot([], [], [], color=body.color, linestyle='-', alpha=0.5)[0]
            for body in self.dataset.moons
        }

        # Test particles (either as individual markers or as a heatmap)
        if not heatmap:
            test_particle_marker = ax.plot([], [], [], ".", label="Test Particles", color="navy", markersize=0.5)[0]
        else:
            # We cannot use a 3D heatmap easily, so let's leave this part out for simplicity
            pass

        # Saturn at origin (barycenter) with an "X" marker
        saturn_marker = ax.plot([0], [0], [0], "x", label="Saturn (Barycenter)", color="yellow", markersize=10)[0]

        def init():
            # Initialize markers and trails to empty
            for marker in moon_markers.values():
                marker.set_data_3d([], [], [])
            for trail in moon_trails.values():
                trail.set_data_3d([], [], [])
            if not heatmap:
                test_particle_marker.set_data_3d([], [], [])
                return list(moon_markers.values()) + [test_particle_marker] + list(moon_trails.values()) + [saturn_marker]
            else:
                return list(moon_markers.values()) + list(moon_trails.values()) + [saturn_marker]

        def update(i):
            # Update the positions and trails for all moons in polar coordinates
            for body, marker in moon_markers.items():
                r, theta = cartesian_to_polar(body.pos[i, 0], body.pos[i, 1])
                z = body.pos[i, 2]  # Take the z-direction (height) from the third dimension

                # Plot only points within the theta range [0, 2pi]
                if theta <= 2 * np.pi:
                    marker.set_data_3d([r], [theta], [z])
                
                # Update the trail of the moons in polar coordinates
                trail_r, trail_theta = cartesian_to_polar(
                    body.pos[max(0, i - trail_length):i + 1, 0],
                    body.pos[max(0, i - trail_length):i + 1, 1]
                )
                trail_z = body.pos[max(0, i - trail_length):i + 1, 2]

                # Plot only trails within the theta range [0, 2pi]
                valid_idx = trail_theta <= 2 * np.pi
                moon_trails[body].set_data_3d(trail_r[valid_idx], trail_theta[valid_idx], trail_z[valid_idx])

            # Plot title:
            plt.title(f"frame {i}")
            
            if not heatmap:
                # Update the test particles in polar coordinates
                r_test, theta_test = cartesian_to_polar(
                    self.dataset.positions[i, self.dataset.num_moons:, 0],
                    self.dataset.positions[i, self.dataset.num_moons:, 1]
                )
                z_test = self.dataset.positions[i, self.dataset.num_moons:, 2]  # Test particles in z-direction

                # Plot only points within the theta range [0, 2pi]
                valid_test_idx = theta_test <= 2 * np.pi
                test_particle_marker.set_data_3d(r_test[valid_test_idx], theta_test[valid_test_idx], z_test[valid_test_idx])

                return list(moon_markers.values()) + [test_particle_marker] + list(moon_trails.values()) + [saturn_marker]
            else:
                return list(moon_markers.values()) + list(moon_trails.values()) + [saturn_marker]

        # Create the animation
        self.animation = anim.FuncAnimation(
            figure,
            update,
            init_func=init,
            frames=np.arange(0, self.dataset.positions.shape[0], interval),
            interval=frame_time,
            blit=False
        )

        # Add a customized legend with the markers outside the plot
        if not heatmap:
            marker_handles = list(moon_markers.values()) + [test_particle_marker, saturn_marker]
        else:
            marker_handles = list(moon_markers.values()) + [saturn_marker]

        figure.legend(handles=marker_handles, loc='center left', bbox_to_anchor=(0.85, 0.5), title="Bodies")

        plt.subplots_adjust(right=0.85)  # Adjust the plot area to make space for the legend

        plt.show()



# Example usage
#filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-06_17-14-48.bin' # good for 2d and 3d
#filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-11_16-54-17.bin'# good for centered plots #small: 2025-01-11_00-13-15, big: 2025-01-11_16-54-17
#filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-11_00-13-15.bin'
#filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-12_22-23-34.bin'
#filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-13_08-41-15.bin'
# filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-14_21-26-21.bin'
#filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-20_10-57-45.bin'
filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-02-01_23-39-59.bin'
#2025-01-13_11-46-03
dataset = Dataset(filepath)
anim_manager = AnimationManager(dataset)

# 2D Animation
anim_manager.plot_2d(n_farthest_filter = 5)

# 3D Animation
#anim_manager.plot_3d()

# Centered Animation
# For a plot centered around "Daphnis" with a square bounds of 1e8 meters
#anim_manager.plot_centered(moon_name="Pan", width=1e7, trail_length=100, interval=1) # To not get dissy one best uses something with low dt and 1 save modularity

# 3D version of Centered
#anim_manager.plot_centered_3d(moon_name="Daphnis", width=4e7, trail_length=100, interval=1) # To not get dissy one best uses something with low dt and 1 save modularity

# Polar plot
#anim_manager.plot_polar_cartesian(r_max =0.7e8, r_min = 1.5e8,theta_max=np.pi/512 ,frame_time=10, trail_length=100, interval=1, heatmap=False)

# Cylindrical plot
#anim_manager.plot_polar_cartesian_with_z(r_max =1.1e8, r_min = 1.4e8, z_max=1e5 , z_min=-1e5,elevation=20,azimuth=90, frame_time=10, trail_length=100, interval=1)

In [5]:
#anim_manager.plot_centered_3d(moon_name="Daphnis", width=4e7, trail_length=100, interval=1) # To not get dissy one best uses something with low dt and 1 save modularity
anim_manager.plot_2d(n_farthest_filter = 5)
#anim_manager.plot_polar_cartesian_with_z(r_max =1.1e8, r_min = 1.4e8, z_max=1e5 , z_min=-1e5,elevation=20,azimuth=90, frame_time=10, trail_length=100, interval=1)

---------------------------------------------------------------------------------------------------------
File Name: simulation 2025-01-06_17-14-48.bin 
Creation Date: 2025-01-06 17:14:48
Moon Names: Saturn, Mimas, Enceladus, Tethys, Dione, Rhea, Titan, Hyperion, Iapetus, Phoebe, Janus, Epimetheus, Helene, Telesto, Calypso, Atlas, Prometheus, Pandora, Pan, Daphnis
Moon Count: 20
Initial Data Folder: initial_data_moon_count_20_start_date_2017-11-18_00-00-00-000_creation_date_2024-12-24_21-27-40
Epoch: [2458075.5]
dt: 600.0
Timesteps: 10000
Number of Test Particles: 10000
Saved Points Modularity: 20
Skipped Timesteps: 0
Inner Radius: 70000000.0
Outer Radius: 140000000.0
Include Particle-Moon Collisions: True
---------------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------------
File Name: simulation 2025-01-11_16-54-17.bin 
Creation Date: 2025-01-11 16:54:17
Moon Names: Saturn, Mimas, Enceladus, Tethys, Dione, Rhea, Titan, Hyperion, Iapetus, Phoebe, Janus, Epimetheus, Helene, Telesto, Calypso, Atlas, Prometheus, Pandora, Pan, Daphnis
Moon Count: 20
Initial Data Folder: initial_data_moon_count_20_start_date_2017-11-18_00-00-00-000_creation_date_2024-12-24_21-27-40
Epoch: [2458075.5]
dt: 600.0
Timesteps: 4000
Number of Test Particles: 100000
Saved Points Modularity: 1
Skipped Timesteps: 3000
Inner Radius: 70000000.0
Outer Radius: 140000000.0
Ring Folder: ring_data_2025-01-09
Include Particle-Moon Collisions: True
Numerical Integrator: Leapfrog
---------------------------------------------------------------------------------------------------------
---------------------------------------------------------------------------------------------------------
File Name: simulation 2025-01-13_08-41-15.bin 
Creation Date: 2025-01-13 08:41:15
Moon Names: Saturn, Mimas, Enceladus, Tethys, Dione, Rhea, Titan, Hyperion, Iapetus, Phoebe, Janus, Epimetheus, Helene, Telesto, Calypso, Atlas, Prometheus, Pandora, Pan, Daphnis
Moon Count: 20
Initial Data Folder: initial_data_moon_count_20_start_date_2017-11-18_00-00-00-000_creation_date_2024-12-24_21-27-40
Epoch: [2458075.5]
dt: 600.0
Timesteps: 100000
Number of Test Particles: 100000
Saved Points Modularity: 1000
Skipped Timesteps: 0
Inner Radius: 116500000.0
Outer Radius: 119000000.0
Ring Folder: ring_data_2025-01-09
Include Particle-Moon Collisions: True
Numerical Integrator: Leapfrog
---------------------------------------------------------------------------------------------------------

In [10]:
import numpy as np
import matplotlib.pyplot as plt

class FourierRings:
    def __init__(self, dataset):
        """
        Initializes the FourierRings class with a given dataset.

        Args:
            dataset (Dataset): The dataset object containing simulation data.
        """
        self.dataset = dataset
        self.time_steps = self.dataset.positions.shape[0]
        self.num_test_particles = self.dataset.num_test_particles

    def cartesian_to_polar(self, x, y):
        """
        Converts Cartesian coordinates (x, y) to polar (r, theta).

        Args:
            x (ndarray): x-coordinate values.
            y (ndarray): y-coordinate values.

        Returns:
            tuple: Polar coordinates (r, theta).
        """
        r = np.sqrt(x ** 2 + y ** 2)
        theta = np.arctan2(y, x)
        return r, np.mod(theta, 2 * np.pi)  # Ensure theta is in the range [0, 2pi]

    def compute_fft(self, data, dt=1):
        """
        Computes the FFT of a dataset.

        Args:
            data (ndarray): The data to apply FFT on.
            dt (float): Time step between data points, default is 1.

        Returns:
            tuple: Frequencies and corresponding FFT values.
        """
        N = len(data)
        freqs = np.fft.fftfreq(N, dt)  # Frequency bins
        fft_values = np.fft.fft(data)   # FFT of the data
        fft_values = fft_values / len(data)
        return freqs, np.abs(fft_values)  # Only return the magnitude

    def analyze_ring_waveform(self):
        """
        Analyzes the waveforms in the test particles' radial distances using FFT.

        Returns:
            None: Displays a plot of the FFT spectrum.
        """
        # Extract radial distances of all test particles across all time steps
        radial_distances = np.zeros((self.time_steps, self.num_test_particles))
        
        for i in range(self.num_test_particles):
            x = self.dataset.positions[:, self.dataset.num_moons + i, 0]
            y = self.dataset.positions[:, self.dataset.num_moons + i, 1]
            radial_distances[:, i] = self.cartesian_to_polar(x, y)[0]

        # Compute the FFT for each test particle's radial distance
        plt.figure(figsize=(10, 6))
        for i in range(self.num_test_particles):
            radial_data = radial_distances[:, i]
            freqs, fft_values = self.compute_fft(radial_data)

            # Plot the FFT results for the test particle
            plt.plot(freqs, fft_values, label=f"Test Particle {i + 1}")

        # Set plot labels and title
        plt.xlabel('Frequency [Hz]')
        plt.ylabel('Amplitude')
        plt.title('FFT of Test Particles\' Radial Distance')
        #plt.legend() not safe at all
        plt.xlim(0.00001, 0.01)
        print(self.dataset.header['dt'])
        plt.show()

# Example usage
filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-11_00-13-15.bin'
dataset = Dataset(filepath)
fourier_rings = FourierRings(dataset)

# Analyze the waveform in the ring using FFT
fourier_rings.analyze_ring_waveform()


600.0


In [12]:
import numpy as np
import matplotlib.pyplot as plt

class FourierRings:
    def __init__(self, dataset):
        """
        Initializes the FourierRings class with a given dataset.

        Args:
            dataset (Dataset): The dataset object containing simulation data.
        """
        self.dataset = dataset
        self.time_steps = self.dataset.positions.shape[0]
        self.num_test_particles = self.dataset.num_test_particles

    def cartesian_to_polar(self, x, y):
        """
        Converts Cartesian coordinates (x, y) to polar (r, theta).

        Args:
            x (ndarray): x-coordinate values.
            y (ndarray): y-coordinate values.

        Returns:
            tuple: Polar coordinates (r, theta).
        """
        r = np.sqrt(x ** 2 + y ** 2)
        theta = np.arctan2(y, x)
        return r, np.mod(theta, 2 * np.pi)  # Ensure theta is in the range [0, 2pi]

    def compute_fft(self, data, dt=1):
        """
        Computes the FFT of a dataset.

        Args:
            data (ndarray): The data to apply FFT on.
            dt (float): Time step between data points, default is 1.

        Returns:
            tuple: Frequencies and corresponding FFT values.
        """
        N = len(data)
        freqs = np.fft.fftfreq(N, dt)  # Frequency bins
        fft_values = np.fft.fft(data)   # FFT of the data
        fft_values = fft_values / len(data)
        return freqs[:N // 2], np.abs(fft_values[:N // 2])  # Only return positive frequencies

    def group_particles(self, radial_distances, num_groups=5):
        """
        Groups test particles based on their mean radial distances.

        Args:
            radial_distances (ndarray): Radial distances of all test particles over time.
            num_groups (int): Number of groups to divide particles into.

        Returns:
            dict: A dictionary where keys are group indices and values are particle indices.
        """
        mean_distances = np.mean(radial_distances, axis=0)
        sorted_indices = np.argsort(mean_distances)
        group_size = self.num_test_particles // num_groups
        
        groups = {}
        for group_idx in range(num_groups):
            start_idx = group_idx * group_size
            end_idx = (group_idx + 1) * group_size
            groups[group_idx] = sorted_indices[start_idx:end_idx]
        
        return groups

    def analyze_ring_waveform_by_group(self, num_groups=5):
        """
        Analyzes the waveforms of radial distances for groups of test particles using FFT.

        Args:
            num_groups (int): Number of groups to divide particles into.

        Returns:
            None: Displays a plot of the FFT spectrum for each group.
        """
        # Extract radial distances of all test particles across all time steps
        radial_distances = np.zeros((self.time_steps, self.num_test_particles))
        
        for i in range(self.num_test_particles):
            x = self.dataset.positions[:, self.dataset.num_moons + i, 0]
            y = self.dataset.positions[:, self.dataset.num_moons + i, 1]
            radial_distances[:, i] = self.cartesian_to_polar(x, y)[0]

        # Group particles based on their mean radial distances
        groups = self.group_particles(radial_distances, num_groups=num_groups)

        plt.figure(figsize=(12, 8))
        for group_idx, particle_indices in groups.items():
            # Compute the mean radial distance for the group
            group_data = np.mean(radial_distances[:, particle_indices], axis=1)
            freqs, fft_values = self.compute_fft(group_data, dt=self.dataset.header['dt'])

            # Plot the FFT for the group
            plt.plot(freqs, fft_values, label=f"Group {group_idx + 1} ({len(particle_indices)} particles)")

        # Set plot labels and title
        plt.xlabel('Frequency [Hz]')
        plt.ylabel('Amplitude')
        plt.title('FFT of Test Particles\' Radial Distance by Group')
        plt.legend()
        plt.xlim(0.00001, 1/(2*((self.dataset.header['dt']))))
        plt.show()

# Example usage
filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-11_00-13-15.bin'
dataset = Dataset(filepath)
fourier_rings = FourierRings(dataset)

# Analyze the waveform in the ring using FFT by groups
fourier_rings.analyze_ring_waveform_by_group(num_groups=5)


In [18]:
import numpy as np
import matplotlib.pyplot as plt

class FourierRings:
    def __init__(self, dataset, num_bins=1000):
        """
        Initializes the FourierRings class with a given dataset and radial bins.

        Args:
            dataset (Dataset): The dataset object containing simulation data.
            num_bins (int): Number of radial bins to compute the density.
        """
        self.dataset = dataset
        self.num_bins = num_bins
        self.time_steps = self.dataset.positions.shape[0]
        self.num_test_particles = self.dataset.num_test_particles

    def cartesian_to_polar(self, x, y):
        """
        Converts Cartesian coordinates (x, y) to polar (r, theta).

        Args:
            x (ndarray): x-coordinate values.
            y (ndarray): y-coordinate values.

        Returns:
            tuple: Polar coordinates (r, theta).
        """
        r = np.sqrt(x ** 2 + y ** 2)
        theta = np.arctan2(y, x)
        return r, np.mod(theta, 2 * np.pi)

    def compute_fft(self, data, dt=1):
        """
        Computes the FFT of a dataset.

        Args:
            data (ndarray): The data to apply FFT on.
            dt (float): Time step between data points, default is 1.

        Returns:
            tuple: Frequencies and corresponding FFT values.
        """
        N = len(data)
        freqs = np.fft.fftfreq(N, d=dt)  # Frequency bins
        fft_values = np.fft.fft(data)    # FFT of the data
        fft_values = fft_values / N      # Normalize the FFT amplitude
        return freqs[:N // 2], np.abs(fft_values[:N // 2])  # Return positive frequencies

    def compute_density(self):
        """
        Computes the density profile of test particles in radial bins over time.

        Returns:
            ndarray: Radial density as a 2D array (time_steps x num_bins).
        """
        # Radial bin edges
        max_radius = 1.5  # Adjust based on the system's scale
        min_radius = 0.5  # Adjust based on the system's scale
        bin_edges = np.linspace(min_radius, max_radius, self.num_bins + 1)
        
        # Initialize density array
        density = np.zeros((self.time_steps, self.num_bins))
        
        for t in range(self.time_steps):
            x = self.dataset.positions[t, self.dataset.num_moons:, 0]
            y = self.dataset.positions[t, self.dataset.num_moons:, 1]
            r = self.cartesian_to_polar(x, y)[0]
            
            # Compute histogram for this time step
            density[t, :], _ = np.histogram(r, bins=bin_edges)
        
        return density, bin_edges

    def analyze_density_waveform(self):
        """
        Analyzes the density waveform by computing FFT of density in each radial bin.

        Returns:
            None: Displays a plot of the FFT spectrum for each radial bin.
        """
        # Compute radial density
        density, bin_edges = self.compute_density()
        bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
        
        plt.figure(figsize=(12, 8))
        
        # Compute FFT for each radial bin
        for i in range(self.num_bins):
            density_time_series = density[:, i]
            freqs, fft_values = self.compute_fft(density_time_series, dt=self.dataset.header['dt'])
            
            # Plot the FFT result
            plt.plot(freqs, fft_values, label=f"Radial Bin {i + 1} ({bin_centers[i]:.2f} AU)")
        
        # Set plot labels and title
        plt.xlabel('Frequency [Hz]')
        plt.ylabel('Amplitude')
        plt.title('FFT of Density Waveform in Radial Bins')
        plt.legend(loc="upper right", fontsize='small', ncol=2)
        plt.xlim(0, 1/(2*((self.dataset.header['dt']))))
        plt.show()

# Example usage
filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-11_16-54-17.bin'
dataset = Dataset(filepath)
fourier_rings = FourierRings(dataset)

# Analyze the density waveform using FFT
fourier_rings.analyze_density_waveform()


In [19]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

class FourierRings:
    def __init__(self, dataset):
        """
        Initializes the FourierRings class with a given dataset.

        Args:
            dataset (Dataset): The dataset object containing simulation data.
        """
        self.dataset = dataset
        self.time_steps = self.dataset.positions.shape[0]
        self.num_test_particles = self.dataset.num_test_particles

    def cartesian_to_polar(self, x, y):
        """
        Converts Cartesian coordinates (x, y) to polar (r, theta).
        """
        r = np.sqrt(x ** 2 + y ** 2)
        theta = np.arctan2(y, x)
        return r, np.mod(theta, 2 * np.pi)  # Ensure theta is in the range [0, 2pi]

    def compute_density(self, num_bins):
        """
        Computes the density in radial bins for each time step.

        Args:
            num_bins (int): Number of radial bins.

        Returns:
            ndarray: 2D array where each row corresponds to density in radial bins at a time step.
        """
        radial_distances = np.zeros((self.time_steps, self.num_test_particles))
        for i in range(self.num_test_particles):
            x = self.dataset.positions[:, self.dataset.num_moons + i, 0]
            y = self.dataset.positions[:, self.dataset.num_moons + i, 1]
            radial_distances[:, i] = self.cartesian_to_polar(x, y)[0]

        # Define radial bins
        r_min, r_max = np.min(radial_distances), np.max(radial_distances)
        radial_bins = np.linspace(r_min, r_max, num_bins + 1)

        # Compute density for each time step
        density = np.zeros((self.time_steps, num_bins))
        for t in range(self.time_steps):
            density[t], _ = np.histogram(radial_distances[t], bins=radial_bins, density=True)

        return density, radial_bins

    def animate_density(self, num_bins=50):
        """
        Animates the density in a polar plot.

        Args:
            num_bins (int): Number of radial bins for density calculation.
        """
        # Compute density
        density, radial_bins = self.compute_density(num_bins)

        # Create theta for polar plot
        theta = np.linspace(0, 2 * np.pi, num_bins, endpoint=False)

        # Set up the polar plot
        fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
        line, = ax.plot([], [], lw=2)
        ax.set_ylim(0, np.max(density))  # Set y-axis limit to maximum density

        def update(frame):
            """
            Update function for animation.
            """
            line.set_data(theta, density[frame])
            return line,

        ani = FuncAnimation(
            fig, update, frames=self.time_steps, interval=100, blit=True
        )

        # Show the animation
        plt.show()

# Example usage
filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-11_00-13-15.bin'
dataset = Dataset(filepath)
fourier_rings = FourierRings(dataset)

# Animate the density waveform
fourier_rings.animate_density(num_bins=50)


In [11]:
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt
import numpy as np


class WaveFrequencyAnalyzer:
    """
    Analyzes the frequency components of the motion of moons and test particles in the dataset.

    Attributes:
        dataset (Dataset): The simulation dataset with positional data for moons and test particles.
        dt (float): The time step between simulation points, extracted from the dataset header.
    """
    def __init__(self, dataset):
        self.dataset = dataset
        self.dt = dataset.header['dt']  # Time step between simulation points

    def _cartesian_to_polar(self, x, y):
        """
        Converts Cartesian coordinates (x, y) to polar coordinates (r, theta).

        Args:
            x (ndarray): x-coordinates.
            y (ndarray): y-coordinates.

        Returns:
            tuple: (r, theta), where r is the radial distance and theta is the angle.
        """
        r = np.sqrt(x ** 2 + y ** 2)
        theta = np.arctan2(y, x)
        return r, np.mod(theta, 2 * np.pi)  # Ensure theta is in [0, 2pi]

    def _fft_analysis(self, time_series):
        """
        Performs FFT analysis on a given time series.

        Args:
            time_series (ndarray): The input time series data.

        Returns:
            tuple: Frequencies and FFT result (amplitude).
        """
        # Number of timesteps
        n = len(time_series)
        # Compute the FFT
        fft_values = fft(time_series)
        # Compute the corresponding frequencies
        frequencies = fftfreq(n, d=self.dt)

        # Return frequencies and the magnitude of the FFT
        return frequencies, np.abs(fft_values)

    def analyze_moon(self, moon_name, plot=True):
        """
        Analyzes the frequency components for a specific moon in the dataset.

        Args:
            moon_name (str): Name of the moon to analyze.
            plot (bool): Whether to plot the results. Default is True.

        Returns:
            dict: Dictionary containing FFT results for r, theta, and z.
        """
        moon = next(moon for moon in self.dataset.moons if moon.name == moon_name)

        # Convert Cartesian coordinates to polar (r, theta) and extract z
        r, theta = self._cartesian_to_polar(moon.pos[:, 0], moon.pos[:, 1])
        z = moon.pos[:, 2]

        # Perform FFT for r, theta, and z
        freq_r, fft_r = self._fft_analysis(r)
        freq_theta, fft_theta = self._fft_analysis(theta)
        freq_z, fft_z = self._fft_analysis(z)

        # Optionally plot the FFT results
        if plot:
            self._plot_fft_results(moon_name, freq_r, fft_r, freq_theta, fft_theta, freq_z, fft_z)

        # Return the results as a dictionary
        return {
            'r': (freq_r, fft_r),
            'theta': (freq_theta, fft_theta),
            'z': (freq_z, fft_z)
        }

    def analyze_test_particles(self, particle_index=0, plot=True):
        """
        Analyzes the frequency components for a specific test particle in the dataset.

        Args:
            particle_index (int): Index of the test particle to analyze.
            plot (bool): Whether to plot the results. Default is True.

        Returns:
            dict: Dictionary containing FFT results for r, theta, and z.
        """
        test_particle_pos = self.dataset.test_particles[particle_index].pos

        # Convert Cartesian coordinates to polar (r, theta) and extract z
        r, theta = self._cartesian_to_polar(test_particle_pos[:, 0], test_particle_pos[:, 1])
        z = test_particle_pos[:, 2]

        # Perform FFT for r, theta, and z
        freq_r, fft_r = self._fft_analysis(r)
        freq_theta, fft_theta = self._fft_analysis(theta)
        freq_z, fft_z = self._fft_analysis(z)

        # Optionally plot the FFT results
        if plot:
            self._plot_fft_results(f"Test Particle {particle_index}", freq_r, fft_r, freq_theta, fft_theta, freq_z, fft_z)

        # Return the results as a dictionary
        return {
            'r': (freq_r, fft_r),
            'theta': (freq_theta, fft_theta),
            'z': (freq_z, fft_z)
        }

    def _plot_fft_results(self, name, freq_r, fft_r, freq_theta, fft_theta, freq_z, fft_z):
        """
        Plots the FFT results for the radial, angular, and z components.

        Args:
            name (str): Name of the body being analyzed.
            freq_r (ndarray): Frequencies for the radial component.
            fft_r (ndarray): FFT amplitude for the radial component.
            freq_theta (ndarray): Frequencies for the angular component.
            fft_theta (ndarray): FFT amplitude for the angular component.
            freq_z (ndarray): Frequencies for the z component.
            fft_z (ndarray): FFT amplitude for the z component.
        """
        fig, ax = plt.subplots(3, 1, figsize=(10, 10))

        # Radial component (r)
        ax[0].plot(freq_r, fft_r, color='b')
        ax[0].set_title(f'FFT of Radial Component (r) for {name}')
        ax[0].set_xlabel('Frequency (Hz)')
        ax[0].set_ylabel('Amplitude')

        # Angular component (theta)
        ax[1].plot(freq_theta, fft_theta, color='g')
        ax[1].set_title(f'FFT of Angular Component (theta) for {name}')
        ax[1].set_xlabel('Frequency (Hz)')
        ax[1].set_ylabel('Amplitude')

        # z component
        ax[2].plot(freq_z, fft_z, color='r')
        ax[2].set_title(f'FFT of Z Component for {name}')
        ax[2].set_xlabel('Frequency (Hz)')
        ax[2].set_ylabel('Amplitude')

        plt.tight_layout()
        plt.show()

# Example usage of WaveFrequencyAnalyzer

# Create the dataset
filepath = 'SaturnModelDatabase/simulation_data/simulation 2025-01-11_00-13-15.bin'
dataset = Dataset(filepath)

# Initialize the analyzer with the dataset
analyzer = WaveFrequencyAnalyzer(dataset)

# Analyze a specific moon (e.g., "Daphnis")
analyzer.analyze_moon(moon_name="Daphnis")

# Analyze a specific test particle (e.g., particle 0)
analyzer.analyze_test_particles(particle_index=0)



{'r': (array([ 0.00000000e+00,  1.66500167e-06,  3.33000333e-06, ...,
         -4.99500500e-06, -3.33000333e-06, -1.66500167e-06]),
  array([1.31211380e+11, 7.99958337e+06, 8.12996036e+06, ...,
         8.51972326e+06, 8.12996036e+06, 7.99958337e+06])),
 'theta': (array([ 0.00000000e+00,  1.66500167e-06,  3.33000333e-06, ...,
         -4.99500500e-06, -3.33000333e-06, -1.66500167e-06]),
  array([3101.29776427,   30.00410198,   30.90164234, ...,   32.28033549,
           30.90164234,   30.00410198])),
 'z': (array([ 0.00000000e+00,  1.66500167e-06,  3.33000333e-06, ...,
         -4.99500500e-06, -3.33000333e-06, -1.66500167e-06]),
  array([1922627.86007745,  716069.70804339,  326688.55803931, ...,
          208022.3730623 ,  326688.55803931,  716069.70804339]))}

In [13]:
import numpy as np
from scipy.fft import fft, fftfreq

class FourierAnalysis:
    """
    Performs Fourier transform analysis on the dataset's test particles
    in cylindrical coordinates (r, theta, z) with respect to time.
    """
    def __init__(self, dataset):
        self.dataset = dataset  # This is the Dataset object
        self.num_timesteps = self.dataset.positions.shape[0]
        self.num_test_particles = self.dataset.num_test_particles
        self.dt = self.dataset.header['dt']  # Time step size from the header
        self.timesteps = np.linspace(0, self.num_timesteps * self.dt, self.num_timesteps)
        
        # Extract test particle positions (timesteps x test particles x 3D positions)
        self.test_particle_positions = self.dataset.positions[:, self.dataset.num_moons:, :3]

    def cartesian_to_cylindrical(self, x, y, z):
        """
        Converts Cartesian coordinates to cylindrical coordinates (r, theta, z).
        """
        r = np.sqrt(x ** 2 + y ** 2)
        theta = np.arctan2(y, x)
        return r, theta, z

    def transform_to_cylindrical(self):
        """
        Converts the entire test particle dataset to cylindrical coordinates.
        Returns:
            ndarray: A 3D array of cylindrical coordinates (timesteps x test particles x (r, theta, z)).
        """
        r = np.sqrt(self.test_particle_positions[:, :, 0] ** 2 + self.test_particle_positions[:, :, 1] ** 2)
        theta = np.arctan2(self.test_particle_positions[:, :, 1], self.test_particle_positions[:, :, 0])
        z = self.test_particle_positions[:, :, 2]  # z remains the same

        return np.stack((r, theta, z), axis=-1)

    def compute_fourier_transform(self):
        """
        Computes the Fourier transform over time for all test particles in cylindrical coordinates.
        Returns:
            dict: A dictionary containing the Fourier transforms and frequencies for (r, theta, z).
        """
        # Convert the particle positions to cylindrical coordinates
        cylindrical_coords = self.transform_to_cylindrical()

        # Fourier transform each of the cylindrical coordinates
        fft_results = {}
        fft_results['r'] = fft(cylindrical_coords[:, :, 0], axis=0)  # Fourier transform for r
        fft_results['theta'] = fft(cylindrical_coords[:, :, 1], axis=0)  # Fourier transform for theta
        fft_results['z'] = fft(cylindrical_coords[:, :, 2], axis=0)  # Fourier transform for z

        # Compute the corresponding frequencies
        freqs = fftfreq(self.num_timesteps, d=self.dt)

        return {
            'frequencies': freqs,
            'fourier_r': fft_results['r'],
            'fourier_theta': fft_results['theta'],
            'fourier_z': fft_results['z']
        }

    def plot_fourier_transform(self, particle_index, max_frequency=None):
        """
        Plots the Fourier transform (amplitude spectrum) for a specific test particle.
        Args:
            particle_index (int): The index of the test particle to plot.
            max_frequency (float): Maximum frequency to display on the plot.
        """
        import matplotlib.pyplot as plt
        
        # Compute the Fourier transforms
        fourier_data = self.compute_fourier_transform()
        freqs = fourier_data['frequencies']
        
        # Filter frequencies if needed
        if max_frequency is not None:
            valid_indices = np.abs(freqs) <= max_frequency
        else:
            valid_indices = np.ones_like(freqs, dtype=bool)  # No filtering
        
        # Extract data for the specific particle
        fourier_r = np.abs(fourier_data['fourier_r'][:, particle_index])
        fourier_theta = np.abs(fourier_data['fourier_theta'][:, particle_index])
        fourier_z = np.abs(fourier_data['fourier_z'][:, particle_index])

        # Plot the amplitude spectra
        plt.figure(figsize=(14, 8))
        
        plt.subplot(311)
        plt.plot(freqs[valid_indices], fourier_r[valid_indices], label='Radial Distance (r)', color='r')
        plt.title(f"Fourier Transform of Particle {particle_index} in Cylindrical Coordinates")
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Amplitude')
        plt.grid(True)
        
        plt.subplot(312)
        plt.plot(freqs[valid_indices], fourier_theta[valid_indices], label='Azimuthal Angle (theta)', color='b')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Amplitude')
        plt.grid(True)
        
        plt.subplot(313)
        plt.plot(freqs[valid_indices], fourier_z[valid_indices], label='Height (z)', color='g')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Amplitude')
        plt.grid(True)

        plt.tight_layout()
        plt.show()

# Example usage:
fourier_analysis = FourierAnalysis(dataset)
fourier_analysis.plot_fourier_transform(particle_index=0, max_frequency=1e-5)


In [17]:
import numpy as np
from scipy.fft import fft, fftfreq

class CollectiveWaveFourier:
    def __init__(self, dataset, r_bins, theta_bins, z_bins):
        self.dataset = dataset
        self.num_timesteps = self.dataset.positions.shape[0]
        self.num_test_particles = self.dataset.num_test_particles
        self.dt = self.dataset.header['dt']  # Time step size from the header
        self.timesteps = np.linspace(0, self.num_timesteps * self.dt, self.num_timesteps)

        # Grid size for the cylindrical coordinates (r, theta, z)
        self.r_bins = r_bins
        self.theta_bins = theta_bins
        self.z_bins = z_bins

        # Extract test particle positions (timesteps x test particles x 3D positions)
        self.test_particle_positions = self.dataset.positions[:, self.dataset.num_moons:, :3]

    def cartesian_to_cylindrical(self, x, y, z):
        """
        Converts Cartesian coordinates to cylindrical coordinates (r, theta, z).
        """
        r = np.sqrt(x ** 2 + y ** 2)
        theta = np.arctan2(y, x)
        return r, theta, z

    def construct_wave_function(self):
        """
        Constructs a wave function representing the particle distribution
        in cylindrical coordinates (r, theta, z) over time.
        
        Returns:
            wave_function: A 4D array (timesteps x r_bins x theta_bins x z_bins) 
            representing the particle density in cylindrical coordinates over time.
        """
        # Initialize the wave function grid
        wave_function = np.zeros((self.num_timesteps, self.r_bins, self.theta_bins, self.z_bins))

        # Get the max values of r, theta, and z for grid scaling
        max_r = np.max(np.sqrt(self.test_particle_positions[:, :, 0]**2 + self.test_particle_positions[:, :, 1]**2))
        max_theta = 2 * np.pi  # Theta is always between -pi and pi
        max_z = np.max(self.test_particle_positions[:, :, 2]) - np.min(self.test_particle_positions[:, :, 2])

        # Define grid edges
        r_edges = np.linspace(0, max_r, self.r_bins + 1)
        theta_edges = np.linspace(-np.pi, np.pi, self.theta_bins + 1)
        z_edges = np.linspace(np.min(self.test_particle_positions[:, :, 2]), np.max(self.test_particle_positions[:, :, 2]), self.z_bins + 1)

        # Convert particle positions to cylindrical and bin them into the grid
        for t in range(self.num_timesteps):
            r, theta, z = self.cartesian_to_cylindrical(self.test_particle_positions[t, :, 0],
                                                        self.test_particle_positions[t, :, 1],
                                                        self.test_particle_positions[t, :, 2])
            # Digitize the positions into the grid
            r_indices = np.digitize(r, r_edges) - 1
            theta_indices = np.digitize(theta, theta_edges) - 1
            z_indices = np.digitize(z, z_edges) - 1

            # Count particles in each (r, theta, z) bin
            for i in range(self.num_test_particles):
                if 0 <= r_indices[i] < self.r_bins and 0 <= theta_indices[i] < self.theta_bins and 0 <= z_indices[i] < self.z_bins:
                    wave_function[t, r_indices[i], theta_indices[i], z_indices[i]] += 1

        return wave_function

    def perform_wave_fourier_transform(self):
        """
        Performs Fourier transform on the wave function over time.
        Returns:
            fourier_result: A 4D array (r_bins x theta_bins x z_bins x frequencies) 
            containing the Fourier transformed data for each spatial grid point.
            frequencies: The frequency array.
        """
        wave_function = self.construct_wave_function()

        # Fourier transform over the time axis (axis=0) for each (r, theta, z) point
        fourier_result = fft(wave_function, axis=0)
        
        # Compute corresponding frequencies
        frequencies = fftfreq(self.num_timesteps, d=self.dt)

        return fourier_result, frequencies

    def plot_wave_fourier_spectrum(self, r_index, theta_index, z_index, max_frequency=None):
        """
        Plots the Fourier transform spectrum for a specific (r, theta, z) grid point.
        """
        import matplotlib.pyplot as plt

        fourier_result, frequencies = self.perform_wave_fourier_transform()

        # Extract the Fourier data for the specific grid point
        wave_spectrum = np.abs(fourier_result[:, r_index, theta_index, z_index])

        # Filter frequencies if necessary
        if max_frequency is not None:
            valid_indices = np.abs(frequencies) <= max_frequency
        else:
            valid_indices = np.ones_like(frequencies, dtype=bool)

        # Plot the spectrum
        plt.figure(figsize=(10, 6))
        plt.plot(frequencies[valid_indices], wave_spectrum[valid_indices])
        plt.title(f"Fourier Transform of Wave Function at (r_bin={r_index}, theta_bin={theta_index}, z_bin={z_index})")
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Amplitude')
        plt.grid(True)
        plt.show()

# Example usage:
collective_wave_fourier = CollectiveWaveFourier(dataset, r_bins=20, theta_bins=10, z_bins=20)
collective_wave_fourier.plot_wave_fourier_spectrum(r_index=10, theta_index=4, z_index=10, max_frequency=1e-7)


In [19]:
import numpy as np
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt

# Mock data representing the particle positions over time (timesteps x particles x (x, y, z))
# Here, I'm generating random data for simplicity, but you'd load your actual data.
timesteps = 100
particles = 1000
positions = np.random.rand(timesteps, particles, 3) * 10  # Replace with your data

# Define grid resolution (bins in r, theta, z)
r_bins = 50
theta_bins = 50
z_bins = 50

# Define a maximum range for cylindrical coordinates (assuming all particles stay within these ranges)
max_r = 10
max_theta = 2 * np.pi
max_z = 10

# Time step size
dt = 0.1  # Change based on your actual time step size
time = np.linspace(0, timesteps * dt, timesteps)

# Function to convert Cartesian to cylindrical coordinates
def cartesian_to_cylindrical(x, y, z):
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)
    return r, theta, z

# Initialize the wave function grid (timesteps x r_bins x theta_bins x z_bins)
wave_function = np.zeros((timesteps, r_bins, theta_bins, z_bins))

# Define the edges of the bins for r, theta, and z
r_edges = np.linspace(0, max_r, r_bins + 1)
theta_edges = np.linspace(-np.pi, np.pi, theta_bins + 1)
z_edges = np.linspace(0, max_z, z_bins + 1)

# Loop over time steps to fill the wave function grid
for t in range(timesteps):
    # Get the particle positions at time t
    x = positions[t, :, 0]
    y = positions[t, :, 1]
    z = positions[t, :, 2]

    # Convert to cylindrical coordinates
    r, theta, z = cartesian_to_cylindrical(x, y, z)

    # Digitize the positions into the bins
    r_indices = np.digitize(r, r_edges) - 1
    theta_indices = np.digitize(theta, theta_edges) - 1
    z_indices = np.digitize(z, z_edges) - 1

    # Count particles in each (r, theta, z) bin
    for i in range(particles):
        if 0 <= r_indices[i] < r_bins and 0 <= theta_indices[i] < theta_bins and 0 <= z_indices[i] < z_bins:
            wave_function[t, r_indices[i], theta_indices[i], z_indices[i]] += 1

# Now we have a "wave function" that represents the particle distribution over time in cylindrical coordinates

# Let's Fourier transform this wave function over time for each spatial location
fourier_transform = np.zeros((r_bins, theta_bins, z_bins, timesteps), dtype=complex)
frequencies = fftfreq(timesteps, d=dt)

# Fourier transform over time for each (r, theta, z) bin
for r_idx in range(r_bins):
    for theta_idx in range(theta_bins):
        for z_idx in range(z_bins):
            fourier_transform[r_idx, theta_idx, z_idx, :] = fft(wave_function[:, r_idx, theta_idx, z_idx])

# Example: Plot the Fourier spectrum at a specific (r, theta, z) bin
r_bin = 25
theta_bin = 25
z_bin = 25

# Get the Fourier amplitude for this spatial bin
spectrum = np.abs(fourier_transform[r_bin, theta_bin, z_bin, :])

# Plot the spectrum
plt.plot(frequencies, spectrum)
plt.title(f"Fourier Spectrum at (r={r_bin}, theta={theta_bin}, z={z_bin})")
plt.xlabel("Frequency")
plt.ylabel("Amplitude")
plt.grid(True)
plt.show()


In [22]:
import numpy as np
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt

# Mock data representing structured particle motion (instead of random)
timesteps = 100
particles = 1000
positions = np.zeros((timesteps, particles, 3))  # Timesteps x Particles x (x, y, z)

# Time step size
dt = 0.1
time = np.linspace(0, timesteps * dt, timesteps)

# Create a structured, periodic motion for the particles
for t in range(timesteps):
    # For simplicity, let's create circular motion in the xy-plane and sinusoidal motion in z
    theta_motion = 2 * np.pi * t / timesteps
    r = 5 + 2 * np.sin(2 * np.pi * t / timesteps)  # Oscillating radius
    theta = np.linspace(0, 2 * np.pi, particles) + theta_motion  # Rotate particles around z-axis
    z = 5 + 2 * np.cos(2 * np.pi * t / timesteps)  # Oscillating z-motion
    
    # Convert cylindrical to Cartesian
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    
    positions[t, :, 0] = x
    positions[t, :, 1] = y
    positions[t, :, 2] = z

# Define grid resolution (bins in r and z)
r_bins = 50
z_bins = 50

# Define a maximum range for cylindrical coordinates (assuming all particles stay within these ranges)
max_r = 10
max_z = 10

# Function to convert Cartesian to cylindrical coordinates
def cartesian_to_cylindrical(x, y, z):
    r = np.sqrt(x**2 + y**2)
    return r, z

# Initialize the wave function grid (timesteps x r_bins x z_bins)
wave_function = np.zeros((timesteps, r_bins, z_bins))

# Define the edges of the bins for r and z
r_edges = np.linspace(0, max_r, r_bins + 1)
z_edges = np.linspace(0, max_z, z_bins + 1)

# Loop over time steps to fill the wave function grid
for t in range(timesteps):
    # Get the particle positions at time t
    x = positions[t, :, 0]
    y = positions[t, :, 1]
    z = positions[t, :, 2]

    # Convert to cylindrical coordinates
    r, z = cartesian_to_cylindrical(x, y, z)

    # Digitize the positions into the bins
    r_indices = np.digitize(r, r_edges) - 1
    z_indices = np.digitize(z, z_edges) - 1

    # Count particles in each (r, z) bin
    for i in range(particles):
        if 0 <= r_indices[i] < r_bins and 0 <= z_indices[i] < z_bins:
            wave_function[t, r_indices[i], z_indices[i]] += 1

# Now we have a "wave function" that represents the particle distribution over time in cylindrical coordinates

# Let's create a 2D visualization of the wave function at a few different time steps
fig, ax = plt.subplots(1, 3, figsize=(18, 5))

for i, t in enumerate([0, timesteps//2, timesteps-1]):
    im = ax[i].imshow(wave_function[t, :, :], extent=[0, max_z, 0, max_r], origin='lower', aspect='auto', cmap='viridis')
    ax[i].set_title(f"Particle Density at Time {t*dt:.2f}")
    ax[i].set_xlabel("z")
    ax[i].set_ylabel("r")
    fig.colorbar(im, ax=ax[i])

plt.tight_layout()
plt.show()

# Let's Fourier transform this wave function over time for each (r, z) bin
fourier_transform = np.zeros((r_bins, z_bins, timesteps), dtype=complex)
frequencies = fftfreq(timesteps, d=dt)

# Fourier transform over time for each (r, z) bin
for r_idx in range(r_bins):
    for z_idx in range(z_bins):
        fourier_transform[r_idx, z_idx, :] = fft(wave_function[:, r_idx, z_idx])

# Plot the Fourier spectrum for a specific (r, z) bin
r_bin = 25  # Mid-point in r
z_bin = 25  # Mid-point in z

# Get the Fourier amplitude for this spatial bin
spectrum = np.abs(fourier_transform[r_bin, z_bin, :])

# Plot the spectrum
plt.plot(frequencies, spectrum)
plt.title(f"Fourier Spectrum at (r={r_bin}, z={z_bin})")
plt.xlabel("Frequency")
plt.ylabel("Amplitude")
plt.grid(True)
plt.show()

# Visualize the Fourier transform for all (r, z) bins at a single frequency component (e.g., the first harmonic)
frequency_bin = 5  # Choose a specific frequency component to visualize
fourier_amplitude_at_frequency = np.abs(fourier_transform[:, :, frequency_bin])

# 2D plot of the Fourier amplitude in (r, z) space
plt.imshow(fourier_amplitude_at_frequency, extent=[0, max_z, 0, max_r], origin='lower', aspect='auto', cmap='plasma')
plt.title(f"Fourier Amplitude at Frequency {frequencies[frequency_bin]:.2f}")
plt.xlabel("z")
plt.ylabel("r")
plt.colorbar(label="Amplitude")
plt.show()


In [21]:
from mpl_toolkits.mplot3d import Axes3D

# Choose a time step to visualize the particle positions
time_step = 50  # Pick any time step to visualize

# Get the particle positions at the chosen time step
x = positions[time_step, :, 0]
y = positions[time_step, :, 1]
z = positions[time_step, :, 2]

# 3D Scatter plot of the particles
fig = plt.figure(figsize=(12, 6))

ax = fig.add_subplot(121, projection='3d')
ax.scatter(x, y, z, c='b', marker='o', s=10)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title(f'3D Particle Positions at Time Step {time_step}')

# 2D scatter plot in the r-z plane (cylindrical coordinates)
r, z_cyl = cartesian_to_cylindrical(x, y, z)
ax2 = fig.add_subplot(122)
ax2.scatter(r, z_cyl, c='r', s=10)
ax2.set_xlabel('r')
ax2.set_ylabel('z')
ax2.set_title(f'2D Projection in r-z Plane at Time Step {time_step}')

plt.tight_layout()
plt.show()

# 2D scatter plot in the x-y plane (top-down view)
plt.scatter(x, y, c='g', s=10)
plt.xlabel('x')
plt.ylabel('y')
plt.title(f'2D Projection in x-y Plane at Time Step {time_step}')
plt.axis('equal')  # Ensure equal scaling on both axes
plt.grid(True)
plt.show()


In [13]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from mpl_toolkits.mplot3d import Axes3D

# Parameters
x_max, y_max = 30, 10  # x_max is now 3 times larger (3 * original x_max)
num_points = 10000  # Number of particles
frames = 100
wave_length = 2 * np.pi
speed = 0.2
amplitude_1 = 0.4  # Amplitude for the first cosine wave
amplitude_2 = 0.3  # Amplitude for the second cosine wave
amplitude_3 = 0.2  # Amplitude for the third cosine wave
num_frequencies = 3  # Number of frequencies in the sum
length_of_x = 1.0  # Controls the "stretch" along the x-axis
y_translation_factor = 0.1  # Controls how much the y-translation happens for each wave

# Define different frequencies for the sum of cosines
frequencies = [1, 2, 3]  # Frequencies of the cosines
phases = np.random.uniform(0, 2 * np.pi, num_frequencies)  # Random phase shifts for each frequency

# Create random initial positions for x, y
x = np.random.uniform(0, x_max, num_points)
y = np.random.uniform(0, y_max, num_points)

# Set up the figure and axis
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(0, x_max)
ax.set_ylim(0, y_max)
ax.set_zlim(-1, 1)

# Initialize scatter plot with empty data
scat = ax.scatter([], [], [], s=5)  # 's' controls the size of the points

def update_wave(frame):
    # Create a sum of cosines with different frequencies and random phases
    z = np.zeros_like(x)
    for i in range(num_frequencies):
        # Introduce a y-dependent phase shift for a slight translation effect in y
        y_shift = y_translation_factor * y * i  # Different shifts for each wave
        z += (amplitude_1 if i == 0 else amplitude_2 if i == 1 else amplitude_3) * \
            np.cos(frequencies[i] * 2 * np.pi * (x - speed * frame) / wave_length + phases[i] + y_shift)

    # Apply length_of_x scaling factor to control the "stretch" of the wave
    z *= length_of_x

    # Update the positions and z-values for scatter plot
    scat._offsets3d = (x, y, z)

    return scat,

# Create the animation
ani = anim.FuncAnimation(fig, update_wave, frames=frames, interval=50, blit=False)

plt.show()


In [6]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as anim
from mpl_toolkits.mplot3d import Axes3D

# Parameters
x_max, y_max = 30, 10  # x_max is now 3 times larger (3 * original x_max)
num_points = 1000  # Number of particles
frames = 100
wave_length = 2 * np.pi
speed = 0.2
amplitude = 1.0  # Amplitude of the wave (adjust this value)
num_frequencies = 3  # Number of frequencies in the sum
length_of_x = 1.0  # Controls the "stretch" along the x-axis

# Define different frequencies for the sum of cosines
frequencies = [1, 2, 3]  # Frequencies of the cosines
phases = np.random.uniform(0, 2 * np.pi, num_frequencies)  # Random phase shifts for each frequency

# Create a meshgrid for x, y values (create a 2D grid of points instead of random positions)
x = np.linspace(0, x_max, num_points)
y = np.linspace(0, y_max, num_points)
X, Y = np.meshgrid(x, y)  # Create a mesh grid from the x, y values

# Set up the figure and axis for both the wave and the Fourier transform
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

# Set up the 3D plot for the wave
ax1 = fig.add_subplot(121, projection='3d')
ax1.set_xlim(0, x_max)
ax1.set_ylim(0, y_max)
ax1.set_zlim(-1, 1)
scat = ax1.scatter([], [], [], s=5)  # 's' controls the size of the points

# Set up the plot for the 2D Fourier transform
ax2.set_xlim(0, x_max)  # X-axis for frequencies in x-direction
ax2.set_ylim(0, y_max)  # Y-axis for frequencies in y-direction
img = ax2.imshow(np.zeros((num_points, num_points)), cmap='viridis', origin='lower', animated=True)

def update_wave(frame):
    # Create a sum of cosines with different frequencies and random phases
    z = np.zeros_like(X)
    for i in range(num_frequencies):
        z += amplitude * np.cos(frequencies[i] * 2 * np.pi * (X - speed * frame) / wave_length + phases[i])

    # Apply length_of_x scaling factor to control the "stretch" of the wave
    z *= length_of_x

    # Update the positions and z-values for scatter plot (3D wave)
    scat._offsets3d = (X.flatten(), Y.flatten(), z.flatten())

    # Compute the 2D Fourier Transform (FFT2) of the wave
    z_fft2 = np.fft.fft2(z)  # 2D FFT of the signal
    z_fft2_magnitude = np.abs(z_fft2)  # Magnitude of the Fourier coefficients

    # Update the Fourier transform plot
    img.set_array(np.log(z_fft2_magnitude + 1))  # Use log scale for better visualization

    return scat, img,

# Create the animation
ani = anim.FuncAnimation(fig, update_wave, frames=frames, interval=50, blit=False)

plt.show()
